In [1]:
%load_ext autoreload
%autoreload 2
import sys 
if '/Users/ericliu/Desktop/Latent-Dirichilet-Allocation' not in sys.path: 
    sys.path.append('/Users/ericliu/Desktop/Latent-Dirichilet-Allocation')
import torch as tr 
import numpy as np 
import pandas as pd 
from collections import defaultdict
from pprint import pprint

/Users/ericliu/Desktop/Latent-Dirichilet-Allocation/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.lda_model import LDASmoothed 
from src.generator import doc_generator 
from src.utils import (
    get_vocab_from_docs, 
    get_np_wct, 
    data_loader,
    text_pipeline, 
    process_documents,
) 
from src.text_pre_processor import (
    remove_accented_chars, 
    remove_special_characters, 
    remove_punctuation,
    remove_extra_whitespace_tabs,
    remove_stopwords,
)
from pprint import pprint 
import copy 

In [3]:
gen = doc_generator(
    M = 3,
    L = 20, 
    topic_prior = tr.tensor([1,1,1,1,1], dtype=tr.double)
)

In [4]:
gen.beta

tensor([[0.1131, 0.0589, 0.0824, 0.1060, 0.0471, 0.0824, 0.0294, 0.1060, 0.0059,
         0.0047, 0.0353, 0.0059, 0.0118, 0.0035, 0.0012, 0.0012, 0.0118, 0.0118,
         0.0236, 0.0236, 0.0177, 0.0177, 0.0041, 0.0047, 0.0353, 0.0059, 0.0118,
         0.0353, 0.0177, 0.0118, 0.0029, 0.0035, 0.0029, 0.0012, 0.0353, 0.0029,
         0.0059, 0.0118, 0.0029, 0.0029],
        [0.0012, 0.0047, 0.0047, 0.0030, 0.0355, 0.0118, 0.0237, 0.0030, 0.0948,
         0.0592, 0.0592, 0.1066, 0.0474, 0.1066, 0.0948, 0.1126, 0.0237, 0.0059,
         0.0059, 0.0118, 0.0178, 0.0178, 0.0041, 0.0047, 0.0047, 0.0237, 0.0118,
         0.0296, 0.0059, 0.0237, 0.0030, 0.0036, 0.0030, 0.0012, 0.0036, 0.0030,
         0.0047, 0.0118, 0.0030, 0.0030],
        [0.0014, 0.0057, 0.0043, 0.0036, 0.0143, 0.0071, 0.0286, 0.0036, 0.0071,
         0.0043, 0.0214, 0.0014, 0.0428, 0.0043, 0.0143, 0.0014, 0.0500, 0.0999,
         0.0999, 0.0571, 0.0857, 0.0857, 0.1285, 0.1285, 0.0057, 0.0014, 0.0143,
         0.0071, 0.0071, 

In [5]:
gen.alpha

Dirichlet(concentration: torch.Size([5]))

In [6]:
gen.theta.shape

torch.Size([3, 5])

In [7]:
gen.theta

tensor([[0.1284, 0.0333, 0.6500, 0.1222, 0.0661],
        [0.3563, 0.0658, 0.1423, 0.2494, 0.1862],
        [0.2088, 0.2163, 0.1503, 0.1728, 0.2519]], dtype=torch.float64)

In [8]:
docs = gen.generate_doc()

Document: 0 | word: 0 -> topic: art -> word: asymmetrical
Document: 0 | word: 1 -> topic: law -> word: bankrupt
Document: 0 | word: 2 -> topic: art -> word: concert
Document: 0 | word: 3 -> topic: science -> word: quantum
Document: 0 | word: 4 -> topic: science -> word: research
Document: 0 | word: 5 -> topic: art -> word: Symmetrical
Document: 0 | word: 6 -> topic: art -> word: Craftsmanship
Document: 0 | word: 7 -> topic: art -> word: content
Document: 0 | word: 8 -> topic: art -> word: physical
Document: 0 | word: 9 -> topic: art -> word: picture
Document 0: asymmetrical bankrupt concert quantum research Symmetrical Craftsmanship content physical picture

Document: 1 | word: 0 -> topic: art -> word: Craftsmanship
Document: 1 | word: 1 -> topic: health -> word: injection
Document: 1 | word: 2 -> topic: science -> word: research
Document: 1 | word: 3 -> topic: science -> word: Craftsmanship
Document: 1 | word: 4 -> topic: science -> word: fever
Document: 1 | word: 5 -> topic: health -

In [9]:
docs

{0: 'asymmetrical bankrupt concert quantum research Symmetrical Craftsmanship content physical picture',
 1: 'Craftsmanship injection research Craftsmanship fever decongestant astrophysics recreation scientst decongestant',
 2: 'asymmetrical Olympic copyright fever contract contract electricity infection fever decongestant'}

In [10]:
#docs_raw_dict, raw_word_2_idx, raw_idx_2_word = data_loader('ap')

In [11]:
result = process_documents(docs, sample=True)

There are 3 documents in the dataset after processing
On average estimated document length is 10.0 words per document after processing
There are 21 unique vocab in the corpus after processing


In [12]:
result['vocab_doc_count_dict']

{'asymmetrical': [1, 0, 1],
 'bankrupt': [1, 0, 0],
 'concert': [1, 0, 0],
 'quantum': [1, 0, 0],
 'research': [1, 1, 0],
 'Symmetrical': [1, 0, 0],
 'Craftsmanship': [1, 2, 0],
 'content': [1, 0, 0],
 'physical': [1, 0, 0],
 'picture': [1, 0, 0],
 'injection': [0, 1, 0],
 'fever': [0, 1, 2],
 'decongestant': [0, 2, 1],
 'astrophysics': [0, 1, 0],
 'recreation': [0, 1, 0],
 'scientst': [0, 1, 0],
 'Olympic': [0, 0, 1],
 'copyright': [0, 0, 1],
 'contract': [0, 0, 2],
 'electricity': [0, 0, 1],
 'infection': [0, 0, 1]}

In [13]:
result['vocab_doc_count_array']

array([[1., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 1., 0.],
       [1., 0., 0.],
       [1., 2., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 2.],
       [0., 2., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 2.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [67]:
lda = LDASmoothed(
    docs = result['documents'],
    num_topics = 5, 
    word_ct_dict = result['vocab_doc_count_dict'], 
    word_ct_array = result['vocab_doc_count_array'],
)
#LDASmoothed(docs, 3, wct_dict)

Topic Dirichlet Prior, Alpha
(5,)

Exchangeable Word Dirichlet Prior, Eta 
1

Var Inf - Word Dirichlet prior, Lambda
(5, 21)

Var Inf - Topic Dirichlet prior, Gamma
(3, 5)

loop phi
looped
double
Var -Inf - Word wise Topic Multinomial/Categorical, Phi
(3, 21, 5)


In [68]:
lda._alpha_

array([1.04708219, 0.98292693, 0.97347267, 0.97347428, 1.16278368])

In [69]:
lda._gamma_

array([[3.04708219, 2.98292693, 2.97347267, 2.97347428, 3.16278368],
       [3.04708219, 2.98292693, 2.97347267, 2.97347428, 3.16278368],
       [3.04708219, 2.98292693, 2.97347267, 2.97347428, 3.16278368]])

In [70]:
lda._lambda_

array([[1.04708219, 0.98292693, 0.97347267, 0.97347428, 1.16278368,
        1.07526208, 0.95052839, 1.05181933, 1.02101821, 0.81760009,
        0.89893338, 1.0283693 , 1.15026429, 0.97429619, 0.94330106,
        1.00778149, 0.9378975 , 0.96782864, 0.9953198 , 0.89475728,
        1.08105987],
       [0.87968664, 0.86986295, 1.01644942, 1.07222286, 1.01387299,
        0.98516565, 0.96690772, 0.95138329, 1.10597141, 1.02937237,
        0.95871485, 0.93060329, 1.05898791, 1.1031785 , 1.09256024,
        0.91521059, 0.96611462, 0.94959045, 0.9782464 , 1.08000428,
        1.13828828],
       [1.03320764, 0.93363961, 0.99309432, 1.16117251, 1.00538214,
        0.96711283, 0.97489658, 1.03274514, 0.91811143, 0.94740917,
        1.09087585, 1.02984852, 1.00638968, 1.09653088, 0.92820856,
        0.96431168, 1.02652255, 1.02295647, 0.86193808, 0.95525935,
        0.98065155],
       [1.03755099, 1.02260049, 0.98925295, 0.81711981, 0.99402204,
        1.00269174, 1.26335968, 0.97758574, 1.0270752

In [71]:
lda.e_step(verbose=True)

Iteration 0, Delta Gamma = inf, the ELBO is -115.63744750138825å
Iteration 0 - > Delta Gamme = 0.3178442465204066
Iteration 1, Delta Gamma = 0.3178442465204066, the ELBO is -111.91368169626378å
Iteration 1 - > Delta Gamme = 0.21250797317005754
Iteration 2, Delta Gamma = 0.21250797317005754, the ELBO is -111.76008449962728å
Iteration 2 - > Delta Gamme = 0.19824287343755292
Iteration 3, Delta Gamma = 0.19824287343755292, the ELBO is -111.72396370870925å
Iteration 3 - > Delta Gamme = 0.19236609371309163
Iteration 4, Delta Gamma = 0.19236609371309163, the ELBO is -111.70340759512314å
Iteration 4 - > Delta Gamme = 0.19013296340249616
Iteration 5, Delta Gamma = 0.19013296340249616, the ELBO is -111.68603932237981å
Iteration 5 - > Delta Gamme = 0.1909166406719254
Iteration 6, Delta Gamma = 0.1909166406719254, the ELBO is -111.6693550645056å
Iteration 6 - > Delta Gamme = 0.19476784773559797
Iteration 7, Delta Gamma = 0.19476784773559797, the ELBO is -111.65242823076å
Iteration 7 - > Delta Gamm

/Users/ericliu/Desktop/Latent-Dirichilet-Allocation/src/lda_model.py:200: UserWarning: Update phi, lambda, gamma: Maximum iteration reached at step 100
  


In [72]:
lda._gamma_

array([[7.94071345, 1.79973397, 1.77342074, 1.77342521, 1.85244637],
       [1.79251984, 1.69350903, 1.66875317, 1.66875738, 8.31620034],
       [1.56461282, 1.62767798, 1.60566658, 1.60567033, 8.73611205]])